# Estimate Pi interactively!

Below you'll see a few widget windows, these run a simulation that estimates the value of "Pi" based on the number of random points that fall within vs. outside of a circle.

Click on the "Simulate" button to run a single simulation. You can run multiple simulations at
once by changing the $N_{simulations}$ slider.

Try playing around with the number of total samples to see how the estimate stabilizes over time!

In [1]:
from bqplot import pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets.widgets import IntSlider, HBox, Button
%matplotlib inline

In [2]:
# New functions

def circ(x, radius=2):
    y = np.sqrt(np.clip(radius**2 - x**2, .0001, None))
    return y


def _simulate_many_pis(n_simulations, n_per_sample):
    n_simulations = np.max([1, n_simulations])
    ratios = []
    for ratio in range(n_simulations):
        sample, inside_dots, ratio, samples_y_curve = _simulate_pi(n_per_sample)
        ratios.append(ratio)
    return ratios, sample, inside_dots, samples_y_curve

def _simulate_pi(n_samples):
    sample = np.random.random((n_samples, 2)) * 2
    inside_dots, ratio, samples_y_curve = calculate_ratio(sample)
    return sample, inside_dots, ratio, samples_y_curve

def calculate_ratio(sample):
    # Calculate the y-value of the curve for each input sample's x value
    samples_y_curve = circ(sample[:, 0])
    inside_dots = samples_y_curve > sample[:, 1]
    
    # Calculate ratio of inside dots to outside dots for this quarter circle
    ratio = 4 * (sum(inside_dots) / len(inside_dots))
    return inside_dots, ratio, samples_y_curve

def _update_plot(change=None):
    n_simulations = simulationslider.value
    n_per_sample = samplesslider.value
    ratios, sample, inside_dots, samples_y_curve = _simulate_many_pis(n_simulations, n_per_sample)
    ratio_data = [(ratio, n_per_sample) for ratio in ratios]
    plot_data['ratios'] += ratio_data
    
    # Update the colored scatterplot
    MAX_PLT = 500  # Otherwise it's way too slow
    colors = np.where(inside_dots, '#41f465', "#f44b42").tolist()
    scat.x = sample[:MAX_PLT, 0]
    scat.y = sample[:MAX_PLT, 1]
    
    scat.colors = colors
    fig_circle.title = 'Ratio of in-to-out dots: {}'.format(ratios[-1])
    
    # Update the scatterplot for ratios
    scatter_y, scatter_x = np.array(plot_data['ratios']).T
    ratio_scatter.x = scatter_x
    ratio_scatter.y = scatter_y

In [3]:
# Set up figure
plt.clear()
plot_data = {'ratios': []}
MAX_SAMPLES = 1000
MAX_SIMULATIONS = 50
width = '350px'

# Draw the circle
fig_circle = plt.figure()
x = np.linspace(0, 5, 1000)
y = circ(x)
half_circle = plt.plot(x=x, y=y, s=[100] * len(y), colors=['black']*len(x))
scat = plt.scatter([], [], options={'s': 1})
plt.xlim(0, 2)
fig_circle.layout.height = width
fig_circle.layout.width = width

# Figure with PI estimation
fig_ratios = plt.figure()
fig_ratios.layout.height = width
fig_ratios.layout.width = width
fig_ratios.title = "Estimation of Pi"

pi_line_x = np.arange(0, MAX_SAMPLES, 100)
line = plt.plot(pi_line_x, [np.pi]*len(pi_line_x), ls='--', colors=['black']*len(pi_line_x))
ratio_scatter = plt.scatter([], [], default_size=10)
plt.xlim(0, MAX_SAMPLES)
plt.ylim(2, 4)

# Slider to control number of samples
samplesslider = IntSlider(value=20, min=10, max=MAX_SAMPLES-1, description="$N_{per\_sample}$")

# Slider to control number of simulations to run
simulationslider = IntSlider(value=1, min=1, max=MAX_SIMULATIONS-1, description="$N_{simulations}$")

# Button to run a simulation
simulatebutton = Button(description="Simulate")
simulatebutton.on_click(_update_plot)

# Initialize our UI
box = HBox([fig_circle, fig_ratios])
box2 = HBox([samplesslider, simulationslider, simulatebutton])
display(box, box2)